In [1]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm_notebook

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, MaxPool2D
from keras.initializers import glorot_normal
from keras.optimizers import Adam
from keras.losses import binary_crossentropy

Using TensorFlow backend.


In [3]:
from keras.utils import to_categorical

Feature Extraction from Pre-trained model

KERAS

In [4]:
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np
from keras.applications.resnet50 import ResNet50

In [7]:
model = VGG19(weights='imagenet', include_top=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [9]:
feature_model = Model(inputs=model.input, outputs=model.get_layer('block3_pool').output)

In [52]:
def get_img_ft(feature_model, img_path):
    img = image.load_img(img_path)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = feature_model.predict(x)
    return features     

In [58]:
img_path = './Dataset/frames/lec1/l1s00004.jpg'

In [66]:
img = image.load_img(img_path)
x = image.img_to_array(img)
print(x.shape)
x = np.expand_dims(x, axis=0)
print(x.shape)
x = preprocess_input(x)
print(x.shape)

(240, 320, 3)
(1, 240, 320, 3)
(1, 240, 320, 3)


In [69]:
features = feature_model.predict(x)

In [74]:
np.squeeze(features).shape

(30, 40, 256)

In [56]:
f = get_img_ft(feature_model, './Dataset/frames/lec1/l1s00004.jpg')

In [57]:
f.reshape()

(1, 30, 40, 256)

PYTORCH

In [24]:
res18_model = models.resnet18(pretrained=True)

In [50]:
# Check parameters of model
for name, child in res18_conv.named_children():
    print(name)
    for name2, params in child.named_parameters():
        print(name, name2)

0
0 weight
1
1 weight
1 bias
2
3
4
4 0.conv1.weight
4 0.bn1.weight
4 0.bn1.bias
4 0.conv2.weight
4 0.bn2.weight
4 0.bn2.bias
4 1.conv1.weight
4 1.bn1.weight
4 1.bn1.bias
4 1.conv2.weight
4 1.bn2.weight
4 1.bn2.bias
5
5 0.conv1.weight
5 0.bn1.weight
5 0.bn1.bias
5 0.conv2.weight
5 0.bn2.weight
5 0.bn2.bias
5 0.downsample.0.weight
5 0.downsample.1.weight
5 0.downsample.1.bias
5 1.conv1.weight
5 1.bn1.weight
5 1.bn1.bias
5 1.conv2.weight
5 1.bn2.weight
5 1.bn2.bias


In [49]:
res18_conv = nn.Sequential(*list(res18_model.children())[:-4])

In [51]:
for param in res18_conv.parameters():
    param.requires_grad = False

In [9]:
# # Change last layer according to number of classes (binary in this case) 
# n_class = 2
# num_ftrs = model_conv.fc.in_features
# model_conv.fc = nn.Linear(num_ftrs, n_class)

# Freeze all layers till Conv2d_4a_3x3
# ct = []
# for name, child in model_conv.named_children():
#     if "Conv2d_4a_3x3" in ct:
#         for params in child.parameters():
#             params.requires_grad = True
#     ct.append(name)

# # To view which layers are freeze and which layers are not freezed:
# for name, child in model_conv.named_children():
#     for name_2, params in child.named_parameters():
#         print(name_2, params.requires_grad)

In [17]:
import os,shutil
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA

In [18]:
from tqdm import tqdm_notebook

In [19]:
cur_dir = os.getcwd()

dataset_dir = os.path.join(cur_dir,"Dataset")

labels_dir = os.path.join(dataset_dir,"labels")
frames_dir = os.path.join(dataset_dir,"frames")

n_lecs = len(os.listdir(labels_dir))

listOfLectures = []

for lec in sorted(os.listdir(frames_dir)):
#     print(lec)
    if lec.startswith('.'):
        continue
    frame_list = []
    lec_path = os.path.join(frames_dir,lec)
    for frame in sorted(os.listdir(lec_path)):
#         print(frame)
        frame_list.append([os.path.join(lec_path,frame),])
    listOfLectures.append(frame_list)
#     print(len(frame_list))

# print(listOfLectures[:2])

lec_num = 0
for csv in sorted(os.listdir(labels_dir)):
    csv_path = os.path.join(labels_dir,csv)
    with open(csv_path,'r') as input:
        data = input.read()
        data = data.split()
        for i in range(len(data)):
            listOfLectures[lec_num][i].append(int(data[i]))
    lec_num+=1

Method 1: Stack across channels

In [12]:
X_train = []
Y_train = []
for i in tqdm_notebook(range(len(listOfLectures))):
    for j in range(2,len(listOfLectures[i])-2):
        im_2 = cv2.resize(cv2.imread(listOfLectures[i][j-2][0],0), (128,128))
        im_1 = cv2.resize(cv2.imread(listOfLectures[i][j-1][0],0), (128,128))
        im_0 = cv2.resize(cv2.imread(listOfLectures[i][j][0],0), (128,128))
        im1 = cv2.resize(cv2.imread(listOfLectures[i][j+1][0],0), (128,128))
        im2 = cv2.resize(cv2.imread(listOfLectures[i][j+2][0],0), (128,128))
        x = np.stack((im_1,im_2,im_0,im1,im2),axis=2)
        X_train.append(x)
        Y_train.append(listOfLectures[i][j][1])    

Method 2: Stack side by side (along the width)

In [27]:
X_train = []
Y_train = []
for i in tqdm_notebook(range(len(listOfLectures))):
    for j in range(2,len(listOfLectures[i])-2):
        im_2 = cv2.resize(cv2.imread(listOfLectures[i][j-2][0],0), (128,128))
        im_1 = cv2.resize(cv2.imread(listOfLectures[i][j-1][0],0), (128,128))
        im_0 = cv2.resize(cv2.imread(listOfLectures[i][j][0],0), (128,128))
        im1 = cv2.resize(cv2.imread(listOfLectures[i][j+1][0],0), (128,128))
        im2 = cv2.resize(cv2.imread(listOfLectures[i][j+2][0],0), (128,128))
        x = np.stack((im_1,im_2,im_0,im1,im2),axis=2).reshape((128,128*5))
        X_train.append(x)
        Y_train.append(listOfLectures[i][j][1])  

In [54]:
X_train = []
Y_train = []
for i in tqdm_notebook(range(len(listOfLectures))):
    for j in tqdm_notebook(range(len(listOfLectures[i]))):
        im = cv2.resize(cv2.imread(listOfLectures[i][j][0],1), (224,224)).T
        output = res50_conv(Variable(torch.tensor(im,dtype=torch.float).reshape(1,3,224,224)))
        X_train.append(im)
        Y_train.append(listOfLectures[i][j][1])  

In [55]:
X_train[0].shape

(3, 224, 224)

In [20]:
Y_train = to_categorical(Y_train)

In [22]:
X_train = np.asarray(X_train)
X_train.shape

(28997, 128, 128, 5)

In [23]:
Y_train.shape

(28997, 2)

In [13]:
num_epochs = 5
num_classes = 2
# batch_size = 100
learning_rate = 0.001

In [14]:
model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(128,128,5), padding='valid'))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
# model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

In [15]:
optim = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=True)

In [16]:
model.compile(loss='binary_crossentropy', optimizer=optim)

In [ ]:
model.fit(X_train, Y_train, nb_epoch=5, validation_split=0.2, verbose = 2)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 23197 samples, validate on 5800 samples
Epoch 1/5
 - 2099s - loss: 6.1290 - val_loss: 0.5307
Epoch 2/5
